# Using a different drawing api 👨‍🎨

The following example shows how `ipysketch` can be used with the [js-draw](https://github.com/personalizedrefrigerator/js-draw) api to access sketches from our notebook.
JS-draw is an existing freehand drawing library for JavaScript and TypeScript.

By extending the [Sketch](https://matthewandretaylor.github.io/ipysketch_lite/ipysketch_lite.html#ipysketch_lite.Sketch) class images can be stored and updated whenever a change is made in the frontend.

In [ ]:
custom_template = """
function render({ model, el }) {
  const script = document.createElement('script');
  script.src = "https://cdn.jsdelivr.net/npm/js-draw@1.20.3/dist/bundle.min.js";
  script.onload = () => {
    const sketch = document.createElement('div');
    sketch.innerHTML = `<div id="editor"></div>`;
    sketch.style.display = 'flex';
    sketch.style.justifyContent = 'center';
    sketch.style.alignItems = 'center';
    el.appendChild(sketch);
    var canvas = document.createElement('canvas');
    var ctx = canvas.getContext('2d');

    const editorElement = sketch.querySelector('#editor');
    editorElement.style.width = '800px';
    editorElement.style.height = '600px';
    const editor = new jsdraw.Editor(editorElement);
    const addToHistory = false;
    editor.dispatch(editor.image.setAutoresizeEnabled(true), addToHistory);
    editor.addToolbar();
    
    function canvasUpload() {
      {canvas_upload}
    }

    function madeEdit() {
        var viewport = editor.image.getImportExportViewport();
        canvas.width = viewport.getScreenRectSize().x;
        canvas.height = viewport.getScreenRectSize().y;
        var renderer = new jsdraw.CanvasRenderer(ctx, viewport);
        editor.image.render(renderer, viewport);
        canvasUpload();
    }

    editorElement.addEventListener('mouseup', madeEdit);
    editorElement.addEventListener('touchend', madeEdit);
  };

  document.head.appendChild(script);
}

export default { render };
"""

from ipysketch_lite import Sketch


class CustomSketch(Sketch):
    """
    This includes a template that allows for using different api's to draw on the sketchpad
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def get_template(self):
        sketch_template = custom_template
        for key, value in self.metadata.items():
            sketch_template = sketch_template.replace(key, str(value))

        return sketch_template

In [ ]:
custom_sketch = CustomSketch()

In [ ]:
custom_sketch.image